# Ex2 - Getting and Knowing your Data

This time we are going to pull data directly from the internet.
Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [1]:
# prompt: import pyspark

!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Basics").getOrCreate()
# now you can use spark session
# for example to read a csv file into a dataframe
# df = spark.read.csv("your_file.csv")



### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv).

In [5]:
from pyspark import SparkFiles
url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv'
spark.sparkContext.addFile(url)



In [8]:
%%sh
wget https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv

--2024-11-05 18:12:50--  https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 364975 (356K) [text/plain]
Saving to: ‘chipotle.tsv’

     0K .......... .......... .......... .......... .......... 14% 2.28M 0s
    50K .......... .......... .......... .......... .......... 28% 13.8M 0s
   100K .......... .......... .......... .......... .......... 42% 6.28M 0s
   150K .......... .......... .......... .......... .......... 56% 16.0M 0s
   200K .......... .......... .......... .......... .......... 70% 26.4M 0s
   250K .......... .......... .......... .......... .......... 84% 8.35M 0s
   300K .......... .......... .......... .......... .......... 98% 48.1M 0s
   350K ......                   

In [55]:
path  = SparkFiles.get('chipotle.tsv')
chipo = spark.read.csv("file://" + path, sep='\t', header=True)
chipo.show()

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                NULL|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
|       5|       1| Chips and Guacamole|           

### Step 3. Assign it to a variable called chipo.

### Step 4. See the first 10 entries

In [11]:
chipo.show(10)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                NULL|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
+--------+--------+--------------------+-----------

### Step 5. What is the number of observations in the dataset?

In [12]:
# Solution 1
rows = chipo.count()
print(rows)


4622


In [ ]:
# Solution 2



### Step 6. What is the number of columns in the dataset?

In [13]:
columns = len(chipo.columns)
print(columns)

5


### Step 7. Print the name of all the columns.

In [14]:
column_names = chipo.columns
print(column_names)

['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']


### Step 9. Which was the most-ordered item?

In [27]:
from pyspark.sql.types import IntegerType
chipo = chipo.withColumn("quantity", chipo["quantity"].cast(IntegerType()))
most_ordered = chipo.groupBy("item_name").sum('quantity').orderBy('sum(quantity)', ascending=False).limit(1)
most_ordered.show()

+------------+-------------+
|   item_name|sum(quantity)|
+------------+-------------+
|Chicken Bowl|          761|
+------------+-------------+



### Step 10. For the most-ordered item, how many items were ordered?

In [36]:
chipo = chipo.withColumn("quantity", chipo["quantity"].cast(IntegerType()))
most_ordered = chipo.groupBy("choice_description").sum('quantity').orderBy('sum(quantity)', ascending=False)
most_ordered2 = most_ordered.where(~most_ordered['choice_description'].contains('NULL')).limit(1)
most_ordered2.show()

+------------------+-------------+
|choice_description|sum(quantity)|
+------------------+-------------+
|       [Diet Coke]|          159|
+------------------+-------------+



### Step 11. What was the most ordered item in the choice_description column?

### Step 12. How many items were orderd in total?

In [37]:
total_items = chipo.select('quantity').agg({'quantity': 'sum'}).collect()[0][0]
print(total_items)

4972


### Step 13. Turn the item price into a float

In [64]:
from pyspark.sql.functions import regexp_replace
chipo = chipo.withColumn('item_price', regexp_replace('item_price', '\\$', ''))
chipo = chipo.withColumn("item_price", chipo["item_price"].cast("float"))
chipo.show(10)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Ch and Fresh Toma...|                NULL|      2.39|
|       1|       1|                Izze|        [Clementine]|      3.39|
|       1|       1|    Nantucket Nectar|             [Apple]|      3.39|
|       1|       1|Ch and Tomatillo-...|                NULL|      2.39|
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|     16.98|
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|     10.98|
|       3|       1|          Side of Ch|                NULL|      1.69|
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|     11.75|
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|      9.25|
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|      9.25|
+--------+--------+--------------------+-----------

#### Step 13.a. Check the item price type

In [65]:
print(chipo.schema['item_price'].dataType)

FloatType()


#### Step 13.b. Create a lambda function and change the type of item price

#### Step 13.c. Check the item price type

### Step 14. How much was the revenue for the period in the dataset?

In [69]:
chipo_new = chipo.withColumn('revenue', chipo.quantity * chipo.item_price)
chipo_new.show(10)

+--------+--------+--------------------+--------------------+----------+------------------+
|order_id|quantity|           item_name|  choice_description|item_price|           revenue|
+--------+--------+--------------------+--------------------+----------+------------------+
|       1|       1|Ch and Fresh Toma...|                NULL|      2.39| 2.390000104904175|
|       1|       1|                Izze|        [Clementine]|      3.39| 3.390000104904175|
|       1|       1|    Nantucket Nectar|             [Apple]|      3.39| 3.390000104904175|
|       1|       1|Ch and Tomatillo-...|                NULL|      2.39| 2.390000104904175|
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|     16.98|33.959999084472656|
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|     10.98|10.979999542236328|
|       3|       1|          Side of Ch|                NULL|      1.69| 1.690000057220459|
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|     11.75|        

In [71]:
print(chipo_new.schema['revenue'].dataType)

DoubleType()


In [72]:
revenue = chipo_new.select('revenue').agg({'revenue': 'sum'}).collect()[0][0]
print(revenue)

39237.01973223686


### Step 15. How many orders were made in the period?

In [74]:
distinct_orders = chipo_new.select('order_id').distinct().agg({'order_id': 'count'}).collect()[0][0]
print(distinct_orders)

1834


### Step 16. What is the average revenue amount per order?

In [75]:
# Solution 1
revenue_per_order = revenue / distinct_orders
print(revenue_per_order)


21.39423104265914


In [ ]:
# Solution 2



### Step 17. How many different items are sold?

In [77]:
distinct_items = chipo_new.select('item_name').distinct().agg({'item_name': 'count'}).collect()[0][0]
print(distinct_items)

50
